In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Generating test data**

In [ ]:
import os
# Define path
test_data_path = "/content/drive/MyDrive/test_data"

# Create the directory if it doesn't exist
os.makedirs(test_data_path, exist_ok=True)

print(f"✅ Directory created or already exists at: {test_data_path}")


✅ Directory created or already exists at: /content/drive/MyDrive/test_data


In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub

# Load YAMNet model
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

# Load your fine-tuned classifier model
classifier = tf.keras.models.load_model('/content/drive/MyDrive/fine_tuned_yamnet_classifier1.keras')

# Label map
label_map = {0: "non-emergency", 1: "emergency"}

# Directory containing test .wav files
test_dir = "/content/drive/MyDrive/test_data_wav"

results = []

for file_name in os.listdir(test_dir):
    if not file_name.endswith(".wav"):
        continue

    file_path = os.path.join(test_dir, file_name)

    try:
        # Load waveform
        waveform, sr = librosa.load(file_path, sr=16000, mono=True)
        waveform = waveform.astype(np.float32)

        # Get embeddings from YAMNet
        scores, embeddings, spectrogram = yamnet_model(waveform)
        embedding = tf.reduce_mean(embeddings, axis=0).numpy().reshape(1, -1)  # Shape: (1, 1024)

        # Predict using your classifier
        pred = classifier.predict(embedding, verbose=0)
        class_idx = int(np.argmax(pred[0]))
        confidence = float(pred[0][class_idx])
        label = label_map[class_idx]

        results.append((file_name, label, confidence))

    except Exception as e:
        print(f"❌ Error with {file_name}: {e}")

# Show predictions
print("\n📊 Prediction Results:")
for fname, label, conf in results:
    print(f"🔍 {fname} → {label} (Confidence: {conf:.2f})")



📊 Prediction Results:
🔍 girl-scream-45657.wav → emergency (Confidence: 1.00)
🔍 help-help-322552.wav → non-emergency (Confidence: 0.89)
🔍 fire-truck-siren-307510.wav → emergency (Confidence: 1.00)
🔍 woman-screaming-sfx-screaming-sound-effect-320169.wav → emergency (Confidence: 1.00)
🔍 police-siren-21498.wav → emergency (Confidence: 0.98)
🔍 air-raid-siren-sound-effect-241383.wav → emergency (Confidence: 0.99)
🔍 medical-ambulance-siren-290716.wav → emergency (Confidence: 1.00)
🔍 human-snore-type-1-265473.wav → non-emergency (Confidence: 1.00)
🔍 human-voice-saying-oh-no-301694.wav → emergency (Confidence: 1.00)
🔍 15-human-body-97360.wav → emergency (Confidence: 0.58)
🔍 human-walking-80552.wav → emergency (Confidence: 0.83)
